In [7]:
import networkx as nx
import freeman as fm
import matplotlib.pyplot as plt
from pprint import pprint
import pymysql
import sys
from collections import defaultdict, OrderedDict
import graph_tool as gt
from graph_tool import centrality, draw
import pandas as pd
import statsmodels as sm

import pickle

from setup import mysql_setup

In [8]:
def get_infos():
    conn = mysql_setup()    
    with conn.cursor() as cursor:
        try:
            cursor.execute('''
                SELECT
                    t.id_track, tempt.id_playlist, t.popularity
                FROM
                    Track t 
                    INNER JOIN Temp_T tempt USING (id_track)
                WHERE
                    t.popularity > 55
                ''')
            ret = cursor.fetchall()
        except pymysql.err.IntegrityError as e:
            print("Erro: não foi possivel dar select em playlist")
            return None
    conn.close()
    return ret

def faz_grafo(dic, ligacoes):
    
    g = gt.Graph(directed=False)
    vp_p = g.new_vertex_property("int")
    vp_l = g.new_vertex_property("string")
    vp_d = g.new_vertex_property("int")
    
    list_v = []
    
    
    for ent in dic:
        v_temp = g.add_vertex()
        list_v.append(v_temp)
        vp_p[v_temp] = ent[1]
        vp_l[v_temp] = ent[0]
        
    keys = list(dic.keys())
    
    for i in range(len(dic)-1): #musicas
        pl_count1 = 0
        for pl1 in dic[keys[i]]: #playlists
            for j in range(len(dic)-1-i): #musicas
                count = 0
                pl_count2 = 0
                for pl2 in dic[keys[j+1+i]]: #playlists
                    if pl1 == pl2:
                        count += 1
                    if count >= ligacoes:
                        g.add_edge(list_v[i], list_v[j+1+i])
                    pl_count2 += 1
                    if pl_count2 >= 30:
                        break
            pl_count1 += 1
            if pl_count1 >= 30:
                break
    
    nodes_to_remove = []
    
    for node in reversed(sorted(g.vertices())):
        degree = g.get_total_degrees([node])
        if degree < 1:
            nodes_to_remove.append(node)
        else:
            vp_d[node] = degree
    
    return g, vp_l, vp_p, vp_d

def get_t_w_tt():
    conn = mysql_setup()
    with conn.cursor() as cursor:
        try:
            cursor.execute('''
                SELECT
                    t.id_track, tt.tag_name
                FROM
                    Track t 
                    INNER JOIN Track_Tag tt USING (id_track)
                WHERE
                    t.popularity > 55
                ''')
            res = cursor.fetchall()
        except pymysql.err.IntegrityError as e:
            print("Erro: não foi possivel criar table temp")
    conn.close()
    
    return res

def analisa_e_salva_xlsx(g, vp_l, vp_p, vp_d, nome):
    vp_b, ep_b = centrality.betweenness(g)
    vp_c = centrality.closeness(g)
    
    lista_l = []
    lista_p = []
    lista_d = []
    lista_b = []
    lista_c = []
    lista_e = []

    for vertex in g.vertices():
        lista_l.append(vp_l[vertex])
        lista_p.append(vp_p[vertex])
        lista_d.append(vp_d[vertex])
        lista_b.append(vp_b[vertex])
        lista_c.append(vp_c[vertex])
        
    listas = [lista_l, lista_p, lista_d, lista_b, lista_c]
    with open(nome+"_listas.pickle", "wb") as fp:
        pickle.dump(listas, fp)
    
    td_listas = list(zip(lista_l, lista_p, lista_d, lista_b, lista_c))
    cols_name = ["id_track", "popularity", "degree", "betweenness", "closeness"]
    df = pd.DataFrame(td_listas, columns=cols_name)
    df.to_excel(nome+".xlsx")

In [7]:
res1 = get_t_w_tt()

In [8]:
count_tag_track = {}
for i in res1:
    if i not in count_tag_track.keys():
        count_tag_track[i] = 1
    else:
        count_tag_track[i] += 1

In [9]:
count_dec_track = {}
for j in count_tag_track:
    if j[0] not in count_dec_track.keys():
        count_dec_track[j[0]] = [0, 0, 0, 0, 0, 0]
    if j[1] == "sixties" or j[1] == "60s" or j[1] == "60's":
        count_dec_track[j[0]][0] += count_tag_track[j]
    elif j[1] == "seventies" or j[1] == "70s" or j[1] == "70's":
        count_dec_track[j[0]][1] += count_tag_track[j]
    elif j[1] == "eighties" or j[1] == "80s" or j[1] == "80's":
        count_dec_track[j[0]][2] += count_tag_track[j]
    elif j[1] == "nineties" or j[1] == "90s" or j[1] == "90's":
        count_dec_track[j[0]][3] += count_tag_track[j]
    elif j[1] == "00s" or j[1] == "00's":
        count_dec_track[j[0]][4] += count_tag_track[j]
    elif j[1] == "10s" or j[1] == "10's":
        count_dec_track[j[0]][5] += count_tag_track[j]

In [10]:
list_60 = []
list_70 = []
list_80 = []
list_90 = []
list_00 = []
list_10 = []

for i in count_dec_track:
    anos = count_dec_track[i]
    max_index = anos.index(max(anos))
    if max_index == 0:
        list_60.append(i)
    elif max_index == 1:
        list_70.append(i)
    elif max_index == 2:
        list_80.append(i)
    elif max_index == 3:
        list_90.append(i)
    elif max_index == 4:
        list_00.append(i)
    elif max_index == 5:
        list_10.append(i)
list_all = [list_60, list_70, list_80, list_90, list_00, list_10]
print(len(list_60), len(list_70), len(list_80), len(list_90), len(list_00), len(list_10))
with open("listas.pickle", "wb") as fp:
    pickle.dump(list_all, fp)

3110 1754 2529 5236 5522 8435


In [11]:
list_60[0]

'00bOhb4584JjyfTiXX81mO'

In [ ]:
infos = get_infos()

In [ ]:
d = defaultdict(list)
for info in infos:
    d[(info[0], info[2])].append(info[1])
    
with open("infos.pickle", "wb") as fp:
    pickle.dump(d, fp)

In [ ]:
j = 0
for i in d:
    print(i, d[i])
    j += 1
    if j > 3:
        break

In [ ]:
d_60 = {}
d_70 = {}
d_80 = {}
d_90 = {}
d_00 = {}
for k in d.keys():
    k_id = k[0]
    if len(d[k]) < 2:
        continue
    if (any(k_id in i for i in list_60)):
        d_60[k] = d[k]
    elif (any(k_id in i for i in list_70)):
        d_70[k] = d[k]
    elif (any(k_id in i for i in list_80)):
        d_80[k] = d[k]
    elif (any(k_id in i for i in list_90)):
        d_90[k] = d[k]
    elif (any(k_id in i for i in list_00)):
        d_00[k] = d[k]
with open("infos2.pickle", "wb") as fp:
    pickle.dump([d_60, d_70, d_80, d_90, d_00], fp)

In [ ]:
j = 0
for i in d_60:
    print(i, d_60[i])
    j += 1
    if j > 3:
        break

In [3]:
with open("listas.pickle", "rb") as fp:
    list_60, list_70, list_80, list_90, list_00, list_10 = pickle.load(fp)
with open("infos.pickle", "rb") as fp:
    d = pickle.load(fp)
with open("infos2.pickle", "rb") as fp:
    d_60, d_70, d_80, d_90, d_00 = pickle.load(fp)

In [48]:
g, vp_l, vp_p, vp_d = faz_grafo(d_60, 1)

In [49]:
vp_b, ep_b = centrality.betweenness(g)

In [50]:
vp_c = centrality.closeness(g)

In [51]:
#vp_evalue, vp_evector = centrality.eigenvector(g)

In [52]:
lista_l = []
lista_p = []
lista_d = []
lista_b = []
lista_c = []
lista_e = []

for vertex in g.vertices():
    lista_l.append(vp_l[vertex])
    lista_p.append(vp_p[vertex])
    lista_d.append(vp_d[vertex])
    lista_b.append(vp_b[vertex])
    lista_c.append(vp_c[vertex])
    #lista_e.append(vp_evector[vertex])

In [53]:
print(len(lista_l), len(lista_p), len(lista_d))
td_listas = list(zip(lista_l, lista_p, lista_d, lista_b, lista_c))

2461 2461 2461


In [54]:
cols_name = ["id_track", "popularity", "degree", "betweenness", "closeness"]
df = pd.DataFrame(td_listas, columns=cols_name)
df.head()

,id_track,popularity,degree,betweenness,closeness
0,00bOhb4584JjyfTiXX81mO,59,745,0.000007,0.406636
1,00FRRwuaJP9KimukvLQCOz,61,470,0.000018,0.404880
2,00hVU6kDP67JHurfwG2dtq,66,32,0.000333,0.331611
3,00meczE1jpLTX0BBzIGrAR,61,18852,0.000839,0.473133
4,00oZhqZIQfL9P5CjOP6JsO,59,10902,0.001215,0.469309


In [55]:
df.to_excel("d_60_1.xlsx")

In [9]:
g, vp_l, vp_p, vp_d = faz_grafo(d_60, 1)
analisa_e_salva_xlsx(g, vp_l, vp_p, vp_d, "d_60_1")

In [ ]:
g, vp_l, vp_p, vp_d = faz_grafo(d_60, 2)
analisa_e_salva_xlsx(g, vp_l, vp_p, vp_d, "d_60_2")

In [10]:
g, vp_l, vp_p, vp_d = faz_grafo(d_70, 1)
analisa_e_salva_xlsx(g, vp_l, vp_p, vp_d, "d_70_1")

In [ ]:
g, vp_l, vp_p, vp_d = faz_grafo(d_70, 2)
analisa_e_salva_xlsx(g, vp_l, vp_p, vp_d, "d_70_2")

In [60]:
g, vp_l, vp_p, vp_d = faz_grafo(d_70, 3)
analisa_e_salva_xlsx(g, vp_l, vp_p, vp_d, "d_70_3.xlsx")

In [11]:
g, vp_l, vp_p, vp_d = faz_grafo(d_80, 1)
analisa_e_salva_xlsx(g, vp_l, vp_p, vp_d, "d_80_1")

In [62]:
g, vp_l, vp_p, vp_d = faz_grafo(d_80, 2)
analisa_e_salva_xlsx(g, vp_l, vp_p, vp_d, "d_80_2.xlsx")

In [63]:
g, vp_l, vp_p, vp_d = faz_grafo(d_80, 3)
analisa_e_salva_xlsx(g, vp_l, vp_p, vp_d, "d_80_3.xlsx")

In [12]:
g, vp_l, vp_p, vp_d = faz_grafo(d_90, 1)
analisa_e_salva_xlsx(g, vp_l, vp_p, vp_d, "d_90_1")

In [75]:
g, vp_l, vp_p, vp_d = faz_grafo(d_90, 2)
analisa_e_salva_xlsx(g, vp_l, vp_p, vp_d, "d_90_2.xlsx")

In [13]:
g, vp_l, vp_p, vp_d = faz_grafo(d_00, 1)
analisa_e_salva_xlsx(g, vp_l, vp_p, vp_d, "d_00_1")

In [76]:
g, vp_l, vp_p, vp_d = faz_grafo(d_00, 2)
analisa_e_salva_xlsx(g, vp_l, vp_p, vp_d, "d_00_2.xlsx")

In [28]:
conn.close()

NameError: name 'conn' is not defined